# <center> EDA-4-10. Практика. Статистические тесты в контексте EDA.

## <center> Исследование данных HR-агенства.

HR-агентство изучает тренды на рынке труда в IT. Компания хочет провести исследование на основе данных о зарплатах в сфере Data Science за 2020–2022 годы. Нам дали некоторые данные, которые и необходимо проанализировать для получение ответов на некоторые вопросы. Немного подробнее о данных:

|*Название столбца*|*Описание*                                                                                                                                                                    |
|:----------------:|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|work_year         |Год, котором была выплачена зарплата.                                            |
|experience_level  |Опыт работы на этой должности в течение года со следующими возможными значениями: <br>  - EN — Entry-level/Junior; <br>  - MI — Mid-level/Intermediate; <br>  - SE — Senior-level/Expert; <br>  - EX — Executive-level/Director.|
|employment_type   |Тип трудоустройства для этой роли: <br> - PT — неполный рабочий день; <br> - FT — полный рабочий день; <br> - CT — контракт; <br> - FL — фриланс.|
|job_title         |Роль, в которой соискатель работал в течение года.|
|salary            |Общая выплаченная валовая сумма заработной платы.|
|salary_currency   |Валюта выплачиваемой заработной платы в виде кода валюты ISO 4217.|
|salary_in_usd     |Зарплата в долларах США (валютный курс, делённый на среднее значение курса доллара США за соответствующий год через fxdata.foorilla.com).|
|employee_residence|Основная страна проживания сотрудника в течение рабочего года в виде кода страны ISO 3166.|
|remote_ratio|Общий объём работы, выполняемой удалённо. Возможные значения: <br> - 0 — удалённой работы нет (менее 20 %); <br> - 50 — частично удалённая работа; <br> - 100 — полностью удалённая работа (более 80 %).|
|company_location|Страна главного офиса работодателя или филиала по контракту в виде кода страны ISO 3166.|
|company_size|Среднее количество людей, работавших в компании в течение года: <br> - S — менее 50 сотрудников (небольшая компания); - M — от 50 до 250 сотрудников (средняя компания); <br> - L — более 250 сотрудников (крупная компания).|

### Цели и задачи работы:

Выяснить, какие факторы влияют на зарплату специалиста Data Scientist.

А также ответить на ключевые вопросы HR-агентства:
- Наблюдается ли ежегодный рост зарплат у специалистов Data Scientist?
- Как соотносятся зарплаты Data Scientist и Data Engineer в 2022 году?
- Как соотносятся зарплаты специалистов Data Scientist в компаниях различных размеров?
- Есть ли связь между наличием должностей Data Scientist и Data Engineer и размером компании?
- Если вы найдёте в данных интересные закономерности, также отметьте их в своём анализе.

Продемонстривать использование разных тестов для проверки статистической значимости сделанных выводов:

1. тесты для количественного признака:
    - для одной выборки;
    - для двух выборок;
    - для нескольких выборок;
2. тест для категориальных признаков.

In [3]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np

from scipy import stats
import statsmodels.api as sm
from statsmodels import stats as sms

import matplotlib.pyplot as plt
import seaborn as sns

# делаем визуальную настройку графиков
sns.set_theme("notebook") 
sns.set_palette("Set2") 

Данные были скопированы из урока, но я продублировал их на [Google Drive](), для удобства.

In [4]:
salaries_data = pd.read_csv('data/ds_salaries.csv')
display(salaries_data)

,Unnamed: 0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


In [5]:
# Задаём уровень значимости
alpha = 0.05
print(f'Уровень значимости alpha = {0.05}.')

Уровень значимости alpha = 0.05.


In [6]:
display(salaries_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607 entries, 0 to 606
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          607 non-null    int64 
 1   work_year           607 non-null    int64 
 2   experience_level    607 non-null    object
 3   employment_type     607 non-null    object
 4   job_title           607 non-null    object
 5   salary              607 non-null    int64 
 6   salary_currency     607 non-null    object
 7   salary_in_usd       607 non-null    int64 
 8   employee_residence  607 non-null    object
 9   remote_ratio        607 non-null    int64 
 10  company_location    607 non-null    object
 11  company_size        607 non-null    object
dtypes: int64(5), object(7)
memory usage: 57.0+ KB


None

In [9]:
#Проверка данных на наличие дубликатов

print('Количество дубликатов:', 
salaries_data[salaries_data.duplicated()].shape[0])

Количество дубликатов: 0


Размерность датасета 607х11. Пропущеных значений и дубликатов нет.

Сами данные и их типы подозрений также не вызывают, кроме столбца remote_ratio, которые хоть и является категориальным, но представлен в числовом формате, несмотря на то, что он может иметь всего 3 значения. Файл с данными небольшой, переводить из Object в Category не вижу смысла.